In [ ]:
from scipy import sparse as sp
from matplotlib import pyplot as plt
import numpy as np
import ed_conserve

In [ ]:
H = ed_conserve.calc_H(N=10, J=1, g=0.1)

In [ ]:
energy, _ = sp.linalg.eigsh(H[(1, 0)], k=1, which="SA")
energy

In [ ]:
Hs = ed_conserve.calc_H(N=14, J=1, g=0.5)

In [ ]:
plt.figure()
n_energies = 5
E: dict[tuple[int, int], np.ndarray] = {}
for qn, H in Hs.items():
    E[qn], _ = sp.linalg.eigsh(H, k=n_energies, which="SA")

ks = sorted(list(set(k for _, k in E.keys())))
k_scaled = [k / 14 * (3.1416) for k in ks]

for i in range(n_energies):
    Es = [[], []]
    for k in ks:
        Es[0].append(E[-1, k][i])
        Es[1].append(E[1, k][i])
    plt.plot(k_scaled, Es[0], label=f"$E_{i}$, $p = -1$")
    plt.plot(k_scaled, Es[1], label=f"$E_{i}$, $p = +1$")


plt.xlabel("k")
plt.ylabel("E")
plt.title("Dispersion relation")
plt.legend()
plt.show()

In [ ]:
block_sizes = []
Ns = range(5, 30)
for N in Ns:
    basis, _ = ed_conserve.calc_basis(N=14)
    block_sizes_k = []
    for _, states in basis.items():
        block_sizes_k.append(len(states))
    block_sizes.append(block_sizes_k)
print(_)

In [ ]:
plt.figure()
for i, N in enumerate(Ns):
    plt.scatter([N for _ in block_sizes[i]], block_sizes[i], c="tab:blue")
plt.xlabel("N")
plt.ylabel("Block size")
plt.yscale("log")
plt.show()

In [ ]:
plt.figure()
_Hs = ed_conserve.calc_H(9, 1, 1)
_Hp = _Hs[(1, 0)].todense().real
_Hm = _Hs[(-1, 0)].todense().real
plt.subplot(1, 2, 1)
plt.imshow(_Hp)
plt.subplot(1, 2, 2)
plt.imshow(_Hm)
plt.show()

In [ ]:
# Test partity eigenvalue calculation
even = [int("110000", 2), int("000000", 2), int("010010", 2), int("111111", 2)]
odd =  [int("111000", 2), int("101100", 2), int("100101", 2), int("000001", 2)]

def parity_LF(s, N):
    return int((-1.)**(-N + bin(s).count('1')))

def translate_LF(s, N):
    """Shift the bits of the state `s` one position to the right (cyclically for N bits)."""
    bs = bin(s)[2:].zfill(N)
    return int(bs[-1] + bs[:-1], base=2)


print("Mine: ")
print("Even:", *(ed_conserve.parity_eigenvalue(i, 6) for i in even))
print("Odd:", *(ed_conserve.parity_eigenvalue(i, 6) for i in odd))

print("LF:")
print("Even:", *(parity_LF(i, 6) for i in even))
print("Odd:", *(parity_LF(i, 6) for i in odd))

print("Should give the same result^")

import time
n_iter = 1000000

print("Time parity")
start = time.perf_counter()
for _ in range(n_iter):
    for i, j in zip(even, odd): 
        ed_conserve.parity_eigenvalue(i, 6)
        ed_conserve.parity_eigenvalue(j, 6)
print(f"Mine took {time.perf_counter() - start :.3f} seconds")

start = time.perf_counter()
for _ in range(n_iter):
    for i, j in zip(even, odd): 
        parity_LF(i, 6)
        parity_LF(j, 6)
print(f"Theirs took {time.perf_counter() - start :.3f} seconds")

print("Time translate")
start = time.perf_counter()
for _ in range(n_iter):
    for i, j in zip(even, odd): 
        ed_conserve.translate(i, 6)
        ed_conserve.translate(j, 6)
print(f"Mine took {time.perf_counter() - start :.3f} seconds")

start = time.perf_counter()
for _ in range(n_iter):
    for i, j in zip(even, odd): 
        translate_LF(i, 6)
        translate_LF(j, 6)
print(f"Theirs took {time.perf_counter() - start :.3f} seconds")


In [ ]:
%matplotlib qt5
def generate_plot(Hs: dict[tuple[int, int], sp.csr_matrix]):
    n_energies = 1
    E: dict[tuple[int, int], np.ndarray] = {}
    for qn, H in Hs.items():
        E[qn], _ = sp.linalg.eigsh(H, k=n_energies, which="SA")

    ks = sorted(list(set(k for _, k in E.keys())))
    k_scaled = [k / 14 * (3.1416) for k in ks]

    for i in range(n_energies):
        Es = [[], []]
        for k in ks:
            Es[0].append(E[(-1, k)][i].copy())
            Es[1].append(E[(1, k)][i].copy())
        plt.plot(k_scaled, Es[0], label=f"$E_{i}$, $p = -1$")
        plt.plot(k_scaled, Es[1], label=f"$E_{i}$, $p = +1$")
    plt.xlabel("k")
    plt.ylabel("E")
    # plt.legend()

plt.figure()

Js = [-1, 1]
gs = [0.5, 1, 1.5]
i = 1
for J in Js:
    for g in gs:
        plt.subplot(2, 3, i)
        Hs = ed_conserve.calc_H(N=14, J=J, g=g)
        generate_plot(Hs)
        plt.title(f"{J = }, {g = }")
        i += 1

plt.suptitle("Dispersion relation")
plt.show()